In [1]:
# Libraries:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import copy
import random

Just using the dataset from the data preparation file

In [ ]:
dataset = pd.read_csv('/Users/charlieunsworth/Documents/Python_coursework/data_py.csv')

Used a manual approach instead of test train split to keep the groups intact

In [3]:
# Splitting into test and train datasets, manually to preserve the groups
X = dataset.drop(['Task ID','Supplier ID','Cost'], axis='columns')
y = dataset['Cost']
Groups = dataset['Task ID']

all_tasks = Groups.unique()
np.random.seed(42)
TestGroup = np.random.choice(all_tasks, size=20, replace=False)

test_loc = dataset['Task ID'].isin(list(TestGroup))

test_ids = dataset[test_loc]['Task ID'].to_numpy()
train_ids = dataset[~test_loc]['Task ID'].to_numpy()

train_tasks = dataset['Task ID'][~test_loc]

X_test = X[test_loc].to_numpy()
y_test = y[test_loc].to_numpy()
X_train = X[~test_loc].to_numpy()
y_train = y[~test_loc].to_numpy()

Fitting a simple Lasso regression model on the training data, R2 is not very impressive but its not negative so its something!

In [ ]:
# 3.3 Fitting a Lasso regression model
from sklearn.linear_model import Lasso

lasso = Lasso(alpha=0.001)
lasso.fit(X_train, y_train)
print(lasso.score(X_test,y_test))
y_pred = ridge.predict(X_test)

0.5383883009865307


Manuel suggested we should be using numpy instead of pandas for the error functions, apart from that I've tried to keep it simple
rmse_calc is just a useful function to compute our RMSE scores, it doesn't go into the models at all

In [5]:
def error_function(real_costs, pred_costs):
    real_min = real_costs.min()
    pred_supplier = pred_costs.argmin()
    pred_min = real_costs[pred_supplier]
    error = real_min - pred_min
    return error

def rmse_calc(error_list):
    error_array = np.array(error_list)
    squared_errors = error_array*error_array
    rss = np.sum(squared_errors)
    value = np.sqrt(rss/len(error_array))
    return value

Error calculation is slightly more involved for the held out part, because the tasks used for testing are all together initially, so we have to split perform it for each task

In [6]:
held_out_errors = []
for i in TestGroup:
    task_trues = y_test[test_ids == i]
    task_preds = y_pred[test_ids == i]
    held_out_errors.append(error_function(task_trues, task_preds))
    print(error_function(task_trues, task_preds))

-0.023997590206996378
-0.01886495931830251
-0.018176695386359243
-0.006408969222048588
-0.011245950423476758
-0.0104726223705211
-0.02762685539882359
-0.033259263796944605
0.0
-0.007138839421870102
-0.025453823409779752
-0.0345263004921727
-0.010408508583130804
-0.019862313243964813
-0.0333538663567397
0.0
-0.010170413261649969
-0.0006968233468558904
-0.05661013824413952
-0.019587667953720522


This won't be the final error, since Alex has some changes he wants to make to feature selection, but the code should all work regardless of the features

In [7]:
print(rmse_calc(held_out_errors))

0.02302972973273373


Here we make a scorer out of our error function, since the validation set is all one task (for each fold) it is fairly simple to apply now

In [ ]:
from sklearn.model_selection import LeaveOneGroupOut, cross_val_score
from sklearn.metrics import make_scorer

logo = LeaveOneGroupOut()
error_scorer = make_scorer(error_function)

lasso_cv = Lasso(alpha=0.001)
cv_scores = cross_val_score(lasso_cv, X_train, y_train, cv=logo, groups=train_tasks, scoring=error_scorer)

I think the error goes up because we're only using part of the training data in each fold, hopefully it's nothing to worry about

In [32]:
print(rmse_calc(cv_scores))

0.032439356868181835


We only see predictive accuracy of any kind with tiny alpha values, but it does form a ranking which is the main thing
Takes about 5 minutes to run for me, so bear that in mind

In [24]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Lasso

lasso_hp = Lasso()
params = {'alpha': np.linspace(0.001, 0.0001, 100)}

grid_search = GridSearchCV(lasso_hp, param_grid=params, scoring=error_scorer, cv=logo, n_jobs=-1)
grid_search.fit(X_train, y_train, groups=train_tasks)
lasso_hp_results = pd.DataFrame(grid_search.cv_results_)

Picks out a very very small alpha value, but at least it picks one

In [25]:
print(grid_search.best_params_)

{'alpha': 0.00037272727272727273}


Next job is report to fit the model for the chosen hyperparameters, report the rmse, copy paste the code for a new regression model, maybe MLP? and get some simple viz of the results